In [3]:
# 載入需要的套件
import numpy as np
import yfinance as yf
import datetime as dt
import tensorflow as tf
import pandas as pd

In [18]:
model = tf.keras.models.load_model("stock-predict.model")

In [5]:
import requests
from bs4 import BeautifulSoup

def getName(stock):
	html = requests.get("https://tw.stock.yahoo.com/quote/" + stock).text
	soup = BeautifulSoup(html, "html.parser")

	return soup.find_all("h1")[1].text

In [6]:
class ProgressBar:
	bar_string_fmt = "\rProgress: [{}{}] {}/{}"
	cnt = 0

	def __init__(self, total, bar_total=20):
		# task 的總數
		self.total = total

		# progress bar 的長度，可依個人喜好設定
		self.bar_total = bar_total

	def update(self, step=1):
		# 更新 progress bar 的進度

		total = self.total
		self.cnt += step

		# bar 的數量
		bar_cnt = (int((self.cnt/total)*self.bar_total))
		# 空白的數量
		space_cnt = self.bar_total - bar_cnt

		# 顯示 progress bar
		# "\r" 的意思代表 replace，print 出來的字串不會印在新的一行而是 replace 原本那行同個位置的字符
		# {:.2%}，表示 format 進來的值會以百分比顯示，並只取到小數點後兩位
		progress = self.bar_string_fmt.format(
			"#" * bar_cnt,
			" " * space_cnt,
			# self.cnt/total,
			self.cnt,
			total
		)

		print(progress, end="")
		
		percent = self.cnt/total
		# 100%
		if percent == 1:
			print("\n")
		elif percent >= 1:
			print("")
			
# if __name__ == '__main__':
#     # 測試
#     total = 10000
#     progress_bar = ProgressBar(total)

#     for _ in range(total):
#         # do something...
#         progress_bar.update()

In [28]:
def percent(x, last_close):
    if x >= last_close:
        add = "+"
    else:
        add = ""
    return f"{add}{(x - last_close) / last_close * 100:.2f}%"

In [32]:
days = 50

DJI_Open = 31909

with open("stock-list.txt") as f:
	stocks = [line.rstrip() for line in f]

df = yf.download("2330.TW", end=dt.date.today(), progress=False)
print(f"Latest: {df.index[-1].date()}")

progress_bar = ProgressBar(len(stocks))

data = []
index = []
for stock in stocks:
	df = yf.download(stock, end=dt.date.today(), progress=False)

	raw = np.array(df)
	x_test = np.append(raw[-days:, :5], DJI_Open).reshape(1, days * 5 + 1)
	# y_test = raw[-1, :5]
	lc = raw[-1, 3] # last close

	# predict
	pred = model.predict(x_test, verbose=0)[0]

	name = getName(stock)
	data.append(list(pred)[:-1] + [percent(pred[1], lc), percent(pred[3], lc)])
	index.append(name)

	progress_bar.update()

pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame(data, index, ["開盤", "最高", "最低", "收盤", "最高漲幅", "收盤漲跌幅"])

Latest: 2023-03-10
Progress: [####################] 4/4



,開盤,最高,最低,收盤,最高漲幅,收盤漲跌幅
大魯閣,19.36,19.75,18.90,19.32,+5.07%,+2.79%
承業醫,67.05,68.54,65.53,67.00,+3.23%,+0.90%
慧洋-KY,66.60,67.55,65.87,66.80,+1.58%,+0.45%
宏達電,62.53,63.42,61.85,62.72,+1.31%,+0.19%
